In [ ]:
import numpy as np

In [ ]:
matr1 = np.random.randint(5, size=(18, 256))
matr2 = np.random.randint(5, size=(256, 240))

In [ ]:
%%time
result_1 = matr1 @ matr2
result_1

CPU times: total: 0 ns
Wall time: 1.99 ms


array([[ 948,  997, 1021, ..., 1052, 1091,  934],
       [1027, 1045, 1061, ..., 1060, 1147, 1053],
       [1112, 1090, 1104, ..., 1035, 1074,  994],
       ...,
       [ 999, 1034, 1093, ..., 1046, 1129, 1002],
       [1073,  994, 1049, ..., 1050, 1105,  993],
       [1071, 1060, 1158, ..., 1135, 1167,  972]])

In [ ]:
def split(matrix1, matrix2):
    if np.max([matrix1.shape[0], matrix1.shape[1], matrix2.shape[1]]) == matrix1.shape[0]:
        if matrix1.shape[0] % 2 != 0:
            matrix1 = np.pad(matrix1,((0,1),(0,0)),mode='constant')
            matrix1 = np.vsplit(matrix1, 2)[0]
            return matrix1, matrix2
        else:
            matrix1 = np.vsplit(matrix1, 2)[0]
            return matrix1, matrix2
    elif np.max([matrix1.shape[0], matrix1.shape[1], matrix2.shape[1]]) == matrix2.shape[1]:
        if matrix2.shape[1] % 2 != 0:
            matrix2 = np.pad(matrix2,((0,0),(0,1)),mode='constant')
            matrix2 = np.hsplit(matrix2, 2)[0]
            return matrix1, matrix2
        else:
            matrix2 = np.hsplit(matrix2, 2)[0]
            return matrix1, matrix2
    else:
        if matrix1.shape[1] % 2 != 0:
            matrix1 = np.pad(matrix1,((0,0),(0,1)),mode='constant')
            matrix1 = np.hsplit(matrix1, 2)[0]
            matrix2 = np.pad(matrix1,((0,1),(0,0)),mode='constant')
            matrix2 = np.vsplit(matrix2, 2)[0]
            return matrix1, matrix2
        else:
            matrix1 = np.hsplit(matrix1, 2)[0]
            matrix2 = np.vsplit(matrix2, 2)[0]
            return matrix1, matrix2

In [ ]:
def search_kernel(matrix1, matrix2, kernel):
    while matrix1.shape[0] > kernel[0] or matrix2.shape[1] > kernel[1]:
        matrix1, matrix2 = split(matrix1, matrix2)
    return matrix1, matrix2

In [ ]:
def slice_matrix_a(kernela, matrix):
    matrix_split = np.vsplit(matrix, 
                             [i for i in range(kernela.shape[0],
                                               (matrix.shape[0]//kernela.shape[1])*kernela.shape[0] + kernela.shape[0],
                                                kernela.shape[0])])
    return matrix_split

In [ ]:
def slice_matrix_b(kernelb, matrix):
    matrix_split = np.hsplit(matrix, 
                             [i for i in range(kernelb.shape[1],
                                               (matrix.shape[1]//kernelb.shape[1])*kernelb.shape[1] + kernelb.shape[1],
                                                kernelb.shape[1])])
    return matrix_split

In [ ]:
def multiply_with_kernel_a(kernel, matrix):
    new_matr_ceil = []
    for matrices in matrix:
        if new_matr_ceil == []:
            new_matr_ceil = mult_with_padding(kernel, matrices, sys_array)
        else:
            new_matr_ceil = np.concatenate((new_matr_ceil, mult_with_padding(kernel, matrices, sys_array)), axis = 1)
    return new_matr_ceil

In [ ]:
def mult_all_kernel_a(kernels, matrices):
    new_matr = []
    for kern in kernels:
        if new_matr == []:
            new_matr = multiply_with_kernel_a(kern, matrices)
        else:
            new_matr = np.concatenate((new_matr, multiply_with_kernel_a(kern, matrices)), axis = 0)
    return new_matr

In [ ]:
def padding_matrix(kernel, count, kernel_size):
    (a,b)=kernel.shape 
    if a < kernel_size[0] and count == 0:
        padding=((0,kernel_size[0] - a),(0,0))
        return np.pad(kernel,padding,mode='constant')
    if a < kernel_size[1] and count == 1:
        padding=((0,0),(0,kernel_size[0] - a))
        return np.pad(kernel,padding,mode='constant')
    return kernel

In [ ]:
def mult_with_padding(matrix1, matrix2, kernel):
    old_shape_first_matrix = matrix1.shape[0]
    matrix1 = padding_matrix(matrix1, 0, kernel)
    old_shape_second_matrix = matrix2.shape[1]
    matrix2 = padding_matrix(matrix2, 1, kernel)
    return (matrix1 @ matrix2)[:old_shape_first_matrix, :old_shape_second_matrix]

In [ ]:
def multiply_matrix(first_matrix, second_matrix, kernel_size):
    kernel_a, kernel_b = search_kernel(first_matrix, second_matrix, kernel_size)
    new = slice_matrix_b(kernel_b, second_matrix)
    kerneli = slice_matrix_a(kernel_a, first_matrix)
    return mult_all_kernel_a(kerneli, new)

In [ ]:
sys_array = (32, 32)

In [ ]:
%%time
result_2 = multiply_matrix(matr1, matr2, sys_array)

CPU times: total: 0 ns
Wall time: 3.99 ms


C:\Users\Dmitry\AppData\Local\Temp\ipykernel_3336\3147510744.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if new_matr_ceil == []:


In [ ]:
assert(result_1.all() == result_2.all())
print('TRUE')

TRUE
